In [ ]:
import math
import random
import itertools
import numpy as np
import numba as nb
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint


#lenght of the walk in 11-dimensional environments
r = 0.01

#Generate values for the 11 parameters using the Muller (1959) method:
def Generate_value(l2,l3,R0,v1,v2,v3,v4,v5,v6,Zn,Zs):
    
    #Generating 11 independent normal deviates
    u1 = np.random.normal(0,0.01)
    u2 = np.random.normal(0,0.01)
    u3 = np.random.normal(0,0.01)
    u4 = np.random.normal(0,0.01)
    u5 = np.random.normal(0,0.01)
    u6 = np.random.normal(0,0.01)
    u7 = np.random.normal(0,0.01)
    u8 = np.random.normal(0,0.01)
    u9 = np.random.normal(0,0.01)
    u10= np.random.normal(0,0.01)
    u11= np.random.normal(0,0.01)
    
    #The length of the vector
    d= math.sqrt((u1**2+u2**2+u3**2+u4**2+u5**2+u6**2+u7**2+u8**2+u9**2+u10**2+u11**2))
    
    #update values
    l2 = abs(r*u1/d + l2)
    l3 = abs(r*u2/d + l3)
    R0 = abs(r*u3/d + R0)
    v1 = abs(r*u4/d + v1)
    v2 = abs(r*u5/d + v2)
    v3 = abs(r*u6/d + v3)
    v4 = abs(r*u7/d + v4)
    v5 = abs(r*u8/d + v5)
    v6 = abs(r*u9/d + v6)
    Zn = abs(r*u10/d + Zn)
    Zs = abs(r*u11/d + Zs)
    
    #Return updated values
    return(l2,l3,R0,v1,v2,v3,v4,v5,v6,Zn,Zs)


#choosing values for bacterial proliferation rate (k0), rate of PG production (a), and PG degradation rate (l1) 

k0 = 0.1; a =2;l1 =0.01;

#choose a small and a large value for each parameter

l2_p =[0.1,3];
l3_p = [0.01,0.5];
R0_p =[0.01,0.5];
v1_p =[0.1,5];
v2_p =[0.1,5];
v3_p =[0.1,5];
v4_p =[0.1,5];
v5_p =[0.1,5];
v6_p =[1,10]
Zn_p =[1,5];
Zs_p =[1,5];

param = [l2_p,l3_p,R0_p,v1_p,v2_p,v3_p,v4_p,v5_p,v6_p,Zn_p,Zs_p]




#Starting point in the 11-dimensional environment 
combination_0 = 0

l2,l3,R0,v1,v2,v3,v4,v5,v6,Zn,Zs = list(list(itertools.product(*param))[combination_0])


#Frequency and amplitude of the sinusoidal function
Frequency = 0.0001
Amplitude = 1




#System of Differential equatioons explaining the activation of the fly immune response
#B  = bacterial concentration 
#G  = PG concentration
#R  = Free Receptor concentration
#X  = Receptor-PG complex concentration
#N  = Relish (NF-kB) concentration
#L  = PGRP-LB concentration
#P  = Pirk concentration
#S  = Repressosome concentration
#A  = AMP concentration
#v1 = Rate of receptor production
#v2 = Rate of Relish production
#v3 = Rate of PGRP-LB production
#v4 = Rate of Pirk production
#v5 = Rate of Repressosome production
#v6 = Rate of AMP production

def system(Var,t):
    B,G,R,X,N,L,P,S,A = Var
    dBdt = Amplitude*np.sin(Frequency*t)**2 + k0*B -A*B
    dGdt = k0*B*a - L*G - R*G + P*X +l3*X - l1*G
    dRdt = R0 + v1*(N/(N+Zn))- P*R - R*G +l3*X - l2*R
    dCdt = R*G - P*X - l3*X
    dNdt = v2*X -l2*N
    dLdt = v3*(N/(N+Zn)) -l2*L
    dPdt = v4*(N/(N+Zn)) -l2*P
    dSdt = v5*(N/(N+Zn)) -l2*S
    dAdt = v6* (N/(N+Zn+(Zn*S/Zs))) - l2*A
    return [dBdt,dGdt,dRdt,dCdt,dNdt,dLdt,dPdt,dSdt,dAdt]


#Inital conditions
Var_zero = [0,0,R0,0,0,0,0,0,0]



#Time over which the system of DE is solved

T = 1000
sample_times = np.arange(T)
t_span =  np.linspace(0, sample_times.max(), 10*len(sample_times))


#Solving system of DE using odeint function
solution = odeint(system, Var_zero,t_span)

#Calculation of fitness for the starting point
F1 = np.exp(-(np.mean(solution[:, 0])+(np.mean(solution[:, 2])+np.mean(solution[:, 4])+np.mean(solution[:, 5])+np.mean(solution[:, 6])+np.mean(solution[:, 7])+np.mean(solution[:, 8]))))




#Dataframe to store values
dfout = pd.DataFrame({"l2": [] , "l3": [],"R0":[], "V1": [], "V2": [], "V3": [], "V4": [], "V5": [], "V6": [],"Zn":[], "Zs":[]  ,"Fitness":[]})

#Storing the first set of values (Index = 0)
Index_val = 0
dfout.at[Index_val,'l2'] = l2
dfout.at[Index_val,'l3'] = l3
dfout.at[Index_val,'R0'] = R0
dfout.at[Index_val,'V1'] = v1
dfout.at[Index_val,'V2'] = v2
dfout.at[Index_val,'V3'] = v3
dfout.at[Index_val,'V4'] = v4
dfout.at[Index_val,'V5'] = v5
dfout.at[Index_val,'V6'] = v6
dfout.at[Index_val,'Zn'] = Zn
dfout.at[Index_val,'Zs'] = Zs
dfout.at[Index_val,'Fitness'] = F1



nstep =10000


for yy in range(nstep):
    
    #Keep the previous values
    l2_0,l3_0,R0_0,v1_0,v2_0,v3_0,v4_0,v5_0,v6_0,Zn_0,Zs_0 = l2,l3,R0,v1,v2,v3,v4,v5,v6,Zn,Zs
    
    #Choose new parameter values
    l2,l3,R0,v1,v2,v3,v4,v5,v6,Zn,Zs = Generate_value(l2,l3,R0,v1,v2,v3,v4,v5,v6,Zn,Zs)
    
    #Solve the system of DE using new parameters
    solution = odeint(system, Var_zero,t_span ,rtol=1e-12, atol=1e-12)
    
    #Calculating the fitness for new parameters
    Fitness =  np.exp(-(np.mean(solution[:, 0])+(np.mean(solution[:, 2])+np.mean(solution[:, 4])+np.mean(solution[:, 5])+np.mean(solution[:, 6])+np.mean(solution[:, 7])+np.mean(solution[:, 8]))))
    
    #Accept parameter valeus if new parameters increase the fitness
    if Fitness > dfout['Fitness'][yy]:
        
        l2,l3,R0,v1,v2,v3,v4,v5,v6,Zn,Zs = l2,l3,R0,v1,v2,v3,v4,v5,v6,Zn,Zs
        
        #update the index value
        Index_val = Index_val+1
        
        #Store new parameter values that increase the fitness
        dfout.at[Index_val,'l2'] = l2
        dfout.at[Index_val,'l3'] = l3
        dfout.at[Index_val,'R0'] = R0
        dfout.at[Index_val,'V1'] = v1
        dfout.at[Index_val,'V2'] = v2
        dfout.at[Index_val,'V3'] = v3
        dfout.at[Index_val,'V4'] = v4
        dfout.at[Index_val,'V5'] = v5
        dfout.at[Index_val,'V6'] = v6
        dfout.at[Index_val,'Zn'] = Zn
        dfout.at[Index_val,'Zs'] = Zs
        dfout.at[Index_val,'Fitness'] = Fitness
        
    #Reject parameters if new parameters do not increase the fitness
    else:
        
        #Keep old parameter values
        l2,l3,R0,v1,v2,v3,v4,v5,v6,Zn,Zs =l2_0,l3_0,R0_0,v1_0,v2_0,v3_0,v4_0,v5_0,v6_0,Zn_0,Zs_0
        
        #update the index value
        Index_val = Index_val+1
        
        #Store old parameter values 
        dfout.at[Index_val,'l2'] = l2
        dfout.at[Index_val,'l3'] = l3
        dfout.at[Index_val,'R0'] = R0
        dfout.at[Index_val,'V1'] = v1
        dfout.at[Index_val,'V2'] = v2
        dfout.at[Index_val,'V3'] = v3
        dfout.at[Index_val,'V4'] = v4
        dfout.at[Index_val,'V5'] = v5
        dfout.at[Index_val,'V6'] = v6
        dfout.at[Index_val,'Zn'] = Zn
        dfout.at[Index_val,'Zs'] = Zs
        dfout.at[Index_val,'Fitness'] = dfout['Fitness'][yy]

#Store data in a CSV file
df = pd.DataFrame(data=dfout)
df.to_csv('Optimum.csv', index=False)
